 # `ecommerce-sql-1` SQL Databse

Panduka Bandara <br>


## Installs

In [ ]:
pip install mysql-connector-python

In [ ]:
pip install pyodbc

**Database Type Determines the Library**

| Database Type                 | Default Port | Python Library Example              | Notes                                                                      |
| ----------------------------- | ------------ | ----------------------------------- | -------------------------------------------------------------------------- |
| **MySQL / MariaDB**           | 3306         | `mysql-connector-python`, `PyMySQL` | Uses MySQL protocol.                                                       |
| **SQL Server / Azure SQL DB** | 1433         | `pyodbc`, `pymssql`                 | Uses TDS (Tabular Data Stream) protocol. MySQL libraries cannot speak TDS. |


> - This project is based on mysql server `ecommerce-sql-1.database.windows.net:1433` is Azure SQL Database, which is Microsoft SQL Server, not MySQL.
> - `mysql-connector-python` cannot connect because it only understands the MySQL protocol on port 3306.



**Why `pyodbc` Works for Azure SQL**

> - `pyodbc` uses ODBC drivers that can communicate with SQL Server’s TDS protocol.
> - Supports encryption, Azure authentication, timeouts, and all SQL Server data types.
> -  Recommended and officially supported by Microsoft for Python → Azure SQL connectivity.


In [ ]:
# To work ,Ensure you have ODBC Driver 18 for SQL Server installed on the system
# Add Microsoft repository
!curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/22.04/prod.list > /etc/apt/sources.list.d/mssql-release.list

# Update packages
!apt-get update

# Install ODBC driver and unixODBC
!ACCEPT_EULA=Y apt-get install -y msodbcsql18 unixodbc-dev

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key is deprecated. Manage keyring files in trusted.gpg.d instead (see apt-key(8)).
100   975  100   975    0     0   5177      0 --:--:-- --:--:-- --:--:--  5158
OK
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    89  100    89    0     0    452      0 --:--:-- --:--:-- --:--:--   454
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://packages.microsoft.com/ubuntu/22.04/prod jammy InRelease [3,632 B]
Get:5 https://r2u.stat.ill

## Establish the connection

In [ ]:
import pyodbc

# Connection details
server = 'tcp:ecommerce-sql-1.database.windows.net,1433'
database = 'ecommerce-sql-db-1'
username = 'admin123'
password = 'ecomdp#@@S'   # Replace with your real password

try:
    # Define connection string
    connection_string = f"""
        DRIVER={{ODBC Driver 18 for SQL Server}};
        SERVER={server};
        DATABASE={database};
        UID={username};
        PWD={password};
        Encrypt=yes;
        TrustServerCertificate=no;
        Connection Timeout=30;
    """

    # Connect to Azure SQL
    conn = pyodbc.connect(connection_string)
    print("Connected to Azure SQL Database")

    # Create a cursor
    cursor = conn.cursor()

    # Run a sample query
    cursor.execute("SELECT DB_NAME();")
    row = cursor.fetchone()
    print("You're connected to database:", row[0])

    # Example: fetch current time
    cursor.execute("SELECT SYSDATETIME();")
    print("Current DB time:", cursor.fetchone()[0])

except Exception as e:
    print("Error:", e)
finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()
        print("Connection closed")

Connected to Azure SQL Database
You're connected to database: ecommerce-sql-db-1
Current DB time: 2025-09-07 20:11:33.918300
Connection closed


## Send the data into the SQL Database

In [ ]:
# Import Data
import pandas as pd
order_payments = pd.read_csv('olist_order_payments_dataset.csv')
order_payments.head(5)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [ ]:
import pandas as pd
import pyodbc

# Connection details (Azure SQL)
server = "ecommerce-sql-1.database.windows.net,1433"
database = "ecommerce-sql-db-1"
username = "admin123"
password = "ecomdp#@@S"

# CSV file path
csv_file_path = "olist_order_payments_dataset.csv"

# Table name
table_name = "order_payments"

# Connection string
conn_str = f"""
DRIVER={{ODBC Driver 18 for SQL Server}};
SERVER={server};
DATABASE={database};
UID={username};
PWD={password};
Encrypt=yes;
TrustServerCertificate=no;
Connection Timeout=30;
"""

try:
    # Step 1: Connect to SQL Server
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    print("Connected to SQL Server successfully!")

    # Step 2: Drop table if exists
    cursor.execute(f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE {table_name};")
    print(f"Table '{table_name}' dropped if it existed.")

    # Step 3: Create table
    create_table_query = f"""
    CREATE TABLE {table_name} (
        order_id NVARCHAR(50),
        payment_sequential INT,
        payment_type NVARCHAR(20),
        payment_installments INT,
        payment_value FLOAT
    );
    """
    cursor.execute(create_table_query)
    print(f"Table '{table_name}' created successfully!")

    # Step 4: Load CSV into pandas DataFrame
    df = pd.read_csv(csv_file_path)
    print("CSV data loaded into pandas DataFrame.")

    # Step 5: Insert data in batches
    batch_size = 500
    total_records = len(df)
    print(f"Starting data insertion in batches of {batch_size} records...")

    for start in range(0, total_records, batch_size):
        end = start + batch_size
        batch = df.iloc[start:end]
        records = [tuple(row) for row in batch.itertuples(index=False, name=None)]

        placeholders = ", ".join("?" * len(batch.columns))
        insert_query = f"INSERT INTO {table_name} VALUES ({placeholders})"
        cursor.fast_executemany = True  # Enable fast insert for large batches
        cursor.executemany(insert_query, records)
        conn.commit()
        print(f"Inserted records {start + 1} to {min(end, total_records)} successfully.")

    print(f"All {total_records} records inserted into '{table_name}'.")

except Exception as e:
    print("Error:", e)

finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()
        print("Connection closed")

Connected to SQL Server successfully!
Table 'order_payments' dropped if it existed.
Table 'order_payments' created successfully!
CSV data loaded into pandas DataFrame.
Starting data insertion in batches of 500 records...
Inserted records 1 to 500 successfully.
Inserted records 501 to 1000 successfully.
Inserted records 1001 to 1500 successfully.
Inserted records 1501 to 2000 successfully.
Inserted records 2001 to 2500 successfully.
Inserted records 2501 to 3000 successfully.
Inserted records 3001 to 3500 successfully.
Inserted records 3501 to 4000 successfully.
Inserted records 4001 to 4500 successfully.
Inserted records 4501 to 5000 successfully.
Inserted records 5001 to 5500 successfully.
Inserted records 5501 to 6000 successfully.
Inserted records 6001 to 6500 successfully.
Inserted records 6501 to 7000 successfully.
Inserted records 7001 to 7500 successfully.
Inserted records 7501 to 8000 successfully.
Inserted records 8001 to 8500 successfully.
Inserted records 8501 to 9000 succes

### Efficient ways to send data

Instead of Manually send data by batches , its possible to send them using `cursor.fast_executemany = True`  which allows bulck insertion through `pyodbc`. <br>
OR `from sqlalchemy import create_engine` can be used


#### `cursor.fast_executemany = True` (pyodbc)

* Only works with pyodbc.
* Optimizes `cursor.executemany()` for **bulk inserts**.
* Great for **row-by-row inserts from Python lists or Pandas**.
* Example:

```python
import pyodbc
import pandas as pd

conn = pyodbc.connect("DRIVER={ODBC Driver 18 for SQL Server};SERVER=...;DATABASE=...;UID=...;PWD=...;Encrypt=yes;")
cursor = conn.cursor()
cursor.fast_executemany = True

df = pd.read_csv("data.csv")
records = [tuple(row) for row in df.itertuples(index=False, name=None)]
placeholders = ", ".join("?" * len(df.columns))
cursor.executemany(f"INSERT INTO table_name VALUES ({placeholders})", records)
conn.commit()
```

> **Pros:** Very fast, direct, fine-grained control. <br>
> **Cons:** Works only with pyodbc, not higher-level ORM.

#### `from sqlalchemy import create_engine` (SQLAlchemy)


* SQLAlchemy provides ORM-style abstraction and *connection pooling.
* Works on top of pyodbc (or other DB drivers) to simplify connections and table operations.
* Can insert Pandas DataFrames directly with `df.to_sql()`.
* Example

```python
from sqlalchemy import create_engine
import pandas as pd
import urllib

server = 'ecommerce-sql-1.database.windows.net'
database = 'ecommerce-sql-db-1'
username = 'admin123'
password = 'ecomdp#@@S'

params = urllib.parse.quote_plus(
    f"DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};Encrypt=yes;TrustServerCertificate=no;"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

df = pd.read_csv("data.csv")
df.to_sql("table_name", engine, if_exists="replace", index=False, chunksize=500)
```
> **Pros:** Very clean, less code, automatic batching (`chunksize`), handles schema automatically.
>  **Cons:** Slightly less control than raw pyodbc; may be slower for extremely large datasets unless tuned.



#### Recommendation

| Scenario                                                       | Use                                     |
| -------------------------------------------------------------- | --------------------------------------- |
| You want **maximum speed with large batches** and full control | `pyodbc + fast_executemany=True`        |
| You want **clean Pandas integration and simpler code**         | `SQLAlchemy + df.to_sql(chunksize=...)` |

---
> Its possible to combine both: SQLAlchemy can use a **pyodbc connection with `fast_executemany=True`** for super-fast `to_sql` inserts.


# ecommerce-mongodb-1 Databse

In [1]:
pip install pandas azure-cosmos

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.2/303.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 14.3 MB/s eta 0:00:00


In [2]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 20.0 MB/s eta 0:00:00


In [7]:
import pandas as pd
from pymongo import MongoClient
from urllib.parse import quote_plus
import uuid

# === CONFIGURATION ===
username = "cosdmin123"
password = "ecomdp#@@S"
database_name = "ecommerce_db"
collection_name = "product_category_translation"

# === ESCAPE USERNAME & PASSWORD FOR URI ===
username_enc = quote_plus(username)
password_enc = quote_plus(password)

# === CONNECTION STRING ===
CONNECTION_STRING = (
    f"mongodb+srv://{username_enc}:{password_enc}"
    "@ecommerce-cosmos-db-1.global.mongocluster.cosmos.azure.com/"
    "?tls=true&authMechanism=SCRAM-SHA-256&retrywrites=false&maxIdleTimeMS=120000"
)

# === CONNECT TO COSMOS DB (Mongo API) ===
try:
    client = MongoClient(CONNECTION_STRING, serverSelectionTimeoutMS=5000)
    server_info = client.server_info()
    print("Connected to Cosmos DB")
    print("Server Info:", server_info)
except Exception as e:
    print("Connection failed:", e)
    exit(1)

# === GET DATABASE & COLLECTION ===
db = client[database_name]
collection = db[collection_name]

# === READ CSV ===
csv_file = "/content/product_category_name_translation.csv"
df = pd.read_csv(csv_file)

# === TRANSFORM CSV ROWS TO DOCUMENTS ===
docs = []
for _, row in df.iterrows():
    doc = {
        "_id": str(uuid.uuid4()),  # Cosmos (Mongo API) requires a unique _id
        "product_category_name": row["product_category_name"],
        "product_category_name_english": row["product_category_name_english"]
    }
    docs.append(doc)

# === INSERT INTO COSMOS DB ===
if docs:
    result = collection.insert_many(docs)
    print(f"Inserted {len(result.inserted_ids)} documents into '{collection_name}' collection.")
else:
    print("No data found in CSV.")


/tmp/ipython-input-2759170125.py:25: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  client = MongoClient(CONNECTION_STRING, serverSelectionTimeoutMS=5000)
/usr/local/lib/python3.12/dist-packages/pymongo/synchronous/mongo_client.py:1758: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  self._resolve_srv()


Connected to Cosmos DB
Server Info: {'version': '8.0.0', 'versionArray': [8, 0, 0, 0], 'bits': 64, 'maxBsonObjectSize': 16777216, 'ok': 1.0}
Inserted 71 documents into 'product_category_translation' collection.


In [8]:
from pymongo import MongoClient
from urllib.parse import quote_plus



# === CONNECTION STRING ===
CONNECTION_STRING = (
    f"mongodb+srv://{username_enc}:{password_enc}"
    "@ecommerce-cosmos-db-1.global.mongocluster.cosmos.azure.com/"
    "?tls=true&authMechanism=SCRAM-SHA-256&retrywrites=false&maxIdleTimeMS=120000"
)

# === CONNECT ===
client = MongoClient(CONNECTION_STRING, serverSelectionTimeoutMS=5000)
db = client[database_name]
collection = db[collection_name]

# === RETRIEVE DOCUMENTS ===
print("Retrieving documents from collection...")
for doc in collection.find().limit(10):   # limit to 10 docs for preview
    print(doc)


/tmp/ipython-input-455099000.py:14: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  client = MongoClient(CONNECTION_STRING, serverSelectionTimeoutMS=5000)
/usr/local/lib/python3.12/dist-packages/pymongo/synchronous/mongo_client.py:1758: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  self._resolve_srv()


✅ Retrieving documents from collection...
{'_id': 'ded6c4b3-8a1b-49bf-8640-26b71c6f0615', 'product_category_name': 'beleza_saude', 'product_category_name_english': 'health_beauty'}
{'_id': 'dddc34d5-3a49-4405-950b-5611440c48b3', 'product_category_name': 'informatica_acessorios', 'product_category_name_english': 'computers_accessories'}
{'_id': 'bd2b6b24-b025-4230-a62e-e292f35502ed', 'product_category_name': 'automotivo', 'product_category_name_english': 'auto'}
{'_id': '14d88c3c-3abe-4cc0-9af0-29b1b4f6708e', 'product_category_name': 'cama_mesa_banho', 'product_category_name_english': 'bed_bath_table'}
{'_id': '00f03fcc-9b69-479d-8c02-09fa6f755118', 'product_category_name': 'moveis_decoracao', 'product_category_name_english': 'furniture_decor'}
{'_id': '9637baa4-dd70-470d-9c56-cea11384a69c', 'product_category_name': 'esporte_lazer', 'product_category_name_english': 'sports_leisure'}
{'_id': '46bc16ff-fa0f-4493-a789-0b1692f5d90b', 'product_category_name': 'perfumaria', 'product_category_